<p style="text-align: center;">SEAS 8510 - Analytical MEthods for ML</p>
<p style="text-align: center;">Homework 3</p>
<p style="text-align: center;">Due Date: April 13, 2024 (9:00am EST)</p>

Your homework 2 should be submitted as a pdf containing your responses, your code, and your printed results where applicable. 

All of the imports

In [21]:
import pandas as pd
import numpy as np
from numpy.linalg import matrix_rank
import datetime as dt
import matplotlib.pyplot as plt
from sympy import *
%matplotlib inline


# Functions

## Reduce a matrix to row echelon form

In [16]:
def row_echelon(A):
    """ Return Row Echelon Form of matrix A """

    # if matrix A has no columns or rows,
    # it is already in REF, so we return itself
    r, c = A.shape
    if r == 0 or c == 0:
        return A
    
    # we search for non-zero element in the first column
    for i in range(len(A)):
        if A[i,0] != 0:
            break
    else:
        # if all elements in the first column is zero,
        # we perform REF on matrix from second column
        B = row_echelon(A[:,1:])
        # and then add the first zero-column back
        return np.hstack([A[:,:1], B])

    # if non-zero element happens not in the first row,
    # we switch rows
    if i > 0:
        ith_row = A[i].copy()
        A[i] = A[0]
        A[0] = ith_row

    # we divide first row by first element in it
    A[0] = A[0] / A[0,0]
    # we subtract all subsequent rows with first row (it has 1 now as first element)
    # multiplied by the corresponding element in the first column
    A[1:] -= A[0] * A[1:,0:1]

    # we perform REF on matrix from second row, from second column
    B = row_echelon(A[1:,1:])

    # we add first row and first (zero) column, and return
    return np.vstack([A[:1], np.hstack([A[1:,:1], B]) ])

## Reduced Row Echelon Form

In [18]:
def reduce_to_row_echelon_form(A):
  """
  This function reduces a matrix to reduced row echelon form using Gaussian Elimination.

  Args:
      A: A square matrix represented as a nested list.

  Returns:
      A new matrix in reduced row echelon form (nested list), or None if the matrix is singular.
  """
  # Error handling for non-square matrices
  if not all(len(row) == len(A) for row in A):
    raise ValueError("Input matrix must be square.")

  # Copy the matrix to avoid modifying the original
  rows = len(A)
  cols = len(A[0])
  B = [[A[i][j] for j in range(cols)] for i in range(rows)]

  # Gaussian Elimination
  for i in range(rows):
    # Find a pivot row with a non-zero leading variable (or swap rows if necessary)
    pivot_row = i
    while B[pivot_row][i] == 0 and pivot_row < rows - 1:
      pivot_row += 1
    if pivot_row == rows - 1 and B[pivot_row][i] == 0:
      return None  # Matrix is singular

    # Swap pivot row with the current row if necessary
    if pivot_row != i:
      B[i], B[pivot_row] = B[pivot_row], B[i]

    # Eliminate elements below the leading variable in the current row
    for j in range(i + 1, rows):
      if B[j][i] != 0:
        factor = B[j][i] / B[i][i]
        for k in range(cols):
          B[j][k] -= factor * B[i][k]

  # Check for leading 1s in the diagonal (reduced row echelon form)
  for i in range(rows):
    if B[i][i] != 1:
      # Normalize the leading variable (avoiding division by zero)
      if B[i][i] != 0:
        factor = 1 / B[i][i]
        for k in range(cols):
          B[i][k] *= factor

  return B

# 2.10
Are the following sets of vectors linearly independent?

## a.

$x_1=
\begin{bmatrix}
2\\
-1\\
3
\end{bmatrix}
,\: \:x_2 =
\begin{bmatrix}
1\\
1\\
-2
\end{bmatrix}
,\: \:x_3 =
\begin{bmatrix}
3\\
-3\\
8
\end{bmatrix}
$

To find this out, we merge them into a matrix and convert it to row echelon form. If there are no zero rows, then the rows and columns are linearly independent.

In [19]:
A = np.array([[2, 1, 3],
              [-1, 1, -3],
              [3, -2, 8]])
print("The vectors combined as A are:")
print(A)
print("A in row echelon format:")
print(row_echelon(A))
print(f"Rank of A is {matrix_rank(A)}")


The vectors combined as A are:
[[ 2  1  3]
 [-1  1 -3]
 [ 3 -2  8]]
A in row echelon format:
[[ 1  0  1]
 [ 0  1 -2]
 [ 0  0  1]]
Rank of A is 3
[[1, 0, 1], [0, 1, -2], [0, 0, 1]]


Manual steps.

$\begin{bmatrix}
 2 &  1 &  3\\
-1 &  1 & -3\\
 3 & -2 &  8
\end{bmatrix}$
<\br>\br>
Add row 2 to row 1
<\br>\br>
$\begin{bmatrix}
 1 &  2 &  0\\
-1 &  1 & -3\\
 3 & -2 &  8
\end{bmatrix}$
<\br>\br>
Add row 1 to row 2
<\br>\br>
$\begin{bmatrix}
 1 &  2 &  0\\
 0 &  3 & -3\\
 3 & -2 &  8
\end{bmatrix}$
<\br>\br>
Subtract three times row 1 from row 3
<\br>\br>
$\begin{bmatrix}
 1 &  2 &  0\\
 0 &  3 & -3\\
 0 & -8 &  8
\end{bmatrix}$
<\br>\br>
Divide row 2 by 3
<\br>\br>
$\begin{bmatrix}
 1 &  2 &  0\\
 0 &  1 & -1\\
 0 & -8 &  8
\end{bmatrix}$
<\br>\br>
Add eight times row 2 to row 3
<\br>\br>
$\begin{bmatrix}
 1 &  2 &  0\\
 0 &  1 & -1\\
 0 &  0 &  0
\end{bmatrix}$


This is multiplying a 3x2 matrix with a 3x3 matrix. Since the inner dimensions are different (2 and 3) they cannot be multiplied.

In [2]:
m1 = np.array([[1,2],[4,5],[7,8]])
m2 = np.array([[1,1,0],[0,1,1],[1,0,1]])

print("m1")
print(m1.shape)
print(m1)
print("m2")
print(m2.shape)
print(m2)

print("m1 X m2")
m3 = np.matmul(m1, m2)
print("Did not work as they are not compatible sizes.")


m1
(3, 2)
[[1 2]
 [4 5]
 [7 8]]
m2
(3, 3)
[[1 1 0]
 [0 1 1]
 [1 0 1]]
m1 X m2


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 2)

My results and Numpy agree that these two matrices cannot be multiplied together.

## b.

$
\begin{bmatrix}
1 & 2 & 3\\
4 & 5 & 6\\
7 & 8 & 9
\end{bmatrix}
\begin{bmatrix}
1 & 1 & 0 \\
0 & 1 & 1 \\
1 & 0 & 1
\end{bmatrix}
$

Here we have two 3x3 matrices which can be multiplied together. The result is a 3x3 matrix.

$
\begin{bmatrix}
1 & 2 & 3\\
4 & 5 & 6\\
7 & 8 & 9
\end{bmatrix}
\begin{bmatrix}
1 & 1 & 0 \\
0 & 1 & 1 \\
1 & 0 & 1
\end{bmatrix}
$
$=$
$
\begin{bmatrix}
1*1 + 2*0 + 3*1 & 1*1 + 2*1 + 3*0 & 1*0 + 2*1 + 3*1 \\
4*1 + 5*0 + 6*1 & 4*1 + 5*1 + 6*0 & 4*0 + 5*1 + 6*1 \\
7*1 + 8*0 + 9*1 & 7*1 + 8*1 + 9*0 & 7*0 + 8*1 + 9*1
\end{bmatrix}
$
$=$
$
\begin{bmatrix}
1 + 0 + 3 & 1 + 2 + 0 & 0 + 2 + 3 \\
4 + 0 + 6 & 4 + 5 + 0 & 0 + 5 + 6 \\
7 + 0 + 9 & 7 + 8 + 0 & 0 + 8 + 9
\end{bmatrix}
$
$=$
$
\begin{bmatrix}
4 & 3 & 5 \\
10 & 9 & 11 \\
16 & 15 & 17
\end{bmatrix}
$

In [3]:
m1 = np.array([[1,2,3],[4,5,6],[7,8,9]])
m2 = np.array([[1,1,0],[0,1,1],[1,0,1]])

print("m1")
print(m1.shape)
print(m1)
print("m2")
print(m2.shape)
print(m2)
print("m1 x m2")
m3 = np.matmul(m1, m2)
print(m3.shape)
print(m3)


m1
(3, 3)
[[1 2 3]
 [4 5 6]
 [7 8 9]]
m2
(3, 3)
[[1 1 0]
 [0 1 1]
 [1 0 1]]
m1 x m2
(3, 3)
[[ 4  3  5]
 [10  9 11]
 [16 15 17]]


# c.

$
\begin{bmatrix}
1 & 1 & 0 \\
0 & 1 & 1 \\
1 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
1 & 2 & 3\\
4 & 5 & 6\\
7 & 8 & 9
\end{bmatrix}
$

Here we have two 3x3 matrices which can be multiplied together. The result is a 3x3 matrix.

$
\begin{bmatrix}
1 & 1 & 0 \\
0 & 1 & 1 \\
1 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
1 & 2 & 3\\
4 & 5 & 6\\
7 & 8 & 9
\end{bmatrix}
$
$=$
$
\begin{bmatrix}
1*1 + 1*4 + 0*7 & 1*2 + 1*5 + 0*8 & 1*3 + 1*6 + 0*9 \\
0*1 + 1*4 + 1*7 & 0*2 + 1*5 + 1*8 & 0*3 + 1*6 + 1*9 \\
1*1 + 0*4 + 1*7 & 1*2 + 0*5 + 1*8 & 1*3 + 0*6 + 1*9
\end{bmatrix}
$
$=$
$
\begin{bmatrix}
1 + 4 + 0 & 2 + 5 + 0 & 3 + 6 + 0 \\
0 + 4 + 7 & 0 + 5 + 8 & 0 + 6 + 9 \\
1 + 0 + 7 & 2 + 0 + 8 & 3 + 0 + 9
\end{bmatrix}
$
$=$
$
\begin{bmatrix}
5 & 7 & 9 \\
11 & 13 & 15 \\
8 & 10 & 12
\end{bmatrix}
$

In [4]:
m1 = np.array([[1,1,0],[0,1,1],[1,0,1]])
m2 = np.array([[1,2,3],[4,5,6],[7,8,9]])

print("m1")
print(m1.shape)
print(m1)
print("m2")
print(m2.shape)
print(m2)
print("m1 x m2")
m3 = np.matmul(m1, m2)
print(m3.shape)
print(m3)

m1
(3, 3)
[[1 1 0]
 [0 1 1]
 [1 0 1]]
m2
(3, 3)
[[1 2 3]
 [4 5 6]
 [7 8 9]]
m1 x m2
(3, 3)
[[ 5  7  9]
 [11 13 15]
 [ 8 10 12]]


# d.

$
\begin{bmatrix}
1 & 2 & 1 & 2 \\
4 & 1 & -1 & 4
\end{bmatrix}
\begin{bmatrix}
0 & 3\\
1 & -1\\
2 & 1\\
5 & 2
\end{bmatrix}
$

Here we have a 2x4 matrix multiplied by a 4x2 matrix which can be multiplied together. The result is a 2x2 matrix.

$
\begin{bmatrix}
1 & 2 & 1 & 2 \\
4 & 1 & -1 & 4
\end{bmatrix}
\begin{bmatrix}
0 & 3\\
1 & -1\\
2 & 1\\
5 & 2
\end{bmatrix}
$
$=$
$
\begin{bmatrix}
1*0 + 2*1 + 1*2 + 2*5 & 1*3 + 2*-1 + 1*1 + 2*2\\
4*0 + 1*1 + -1*2 + 4*5 & 4*3 + 1*-1 + -1*1 + 4*2
\end{bmatrix}
$
$=$
$
\begin{bmatrix}
0 + 2 + 2 + 10 & 3 + -2 + 1 + 4\\
0 + 1 + -2 + 20 & 12 + -1 + -1 + 8
\end{bmatrix}
$
$=$
$
\begin{bmatrix}
14 & 6\\
19 & 18
\end{bmatrix}
$

In [5]:
m1 = np.array([[1,2,1,2],[4,1,-1,4]])
m2 = np.array([[0,3],[1,-1],[2,1],[5,2]])

print("m1")
print(m1.shape)
print(m1)
print("m2")
print(m2.shape)
print(m2)
print("m1 x m2")
m3 = np.matmul(m1, m2)
print(m3.shape)
print(m3)

m1
(2, 4)
[[ 1  2  1  2]
 [ 4  1 -1  4]]
m2
(4, 2)
[[ 0  3]
 [ 1 -1]
 [ 2  1]
 [ 5  2]]
m1 x m2
(2, 2)
[[14  6]
 [19 18]]


# e.

$
\begin{bmatrix}
0 & 3\\
1 & -1\\
2 & 1\\
5 & 2
\end{bmatrix}
\begin{bmatrix}
1 & 2 & 1 & 2 \\
4 & 1 & -1 & 4
\end{bmatrix}
$

Here we have a 4x2 matrix multiplied by a 2x4 matrix which can be multiplied together. The result is a 4x4 matrix.

$
\begin{bmatrix}
0 & 3\\
1 & -1\\
2 & 1\\
5 & 2
\end{bmatrix}
\begin{bmatrix}
1 & 2 & 1 & 2 \\
4 & 1 & -1 & 4
\end{bmatrix}
$
$=$
$
\begin{bmatrix}
0*1 + 3*4 & 0*2 + 3*1 & 0*1 + 3*-1 & 0*2 + 3*4\\
1*1 + -1*4 & 1*2 + -1*1 & 1*1 + -1*-1 & 1*2 + -1*4\\
2*1 + 1*4 & 2*2 + 1*1 & 2*1 + 1*-1 & 2*2 + 1*4\\
5*1 + 2*4 & 5*2 + 2*1 & 5*1 + 2*-1 & 5*2 + 2*4\\
\end{bmatrix}
$
$=$
$
\begin{bmatrix}
0 + 12 & 0 + 3 & 0 + -3 & 0 + 12\\
1 + -4 & 2 + -1 & 1 + 1 & 2 + -4\\
2 + 4 & 4 + 1 & 2 + -1 & 4 + 4\\
5 + 8 & 10 + 2 & 5 + -2 & 10 + 8
\end{bmatrix}
$
$=$
$
\begin{bmatrix}
12 & 3 & -3 & 12\\
-3 & 1 & 2 & -2\\
6 & 5 & 1 & 8\\
13 & 12 & 3 & 18
\end{bmatrix}
$

In [6]:
m1 = np.array([[0,3],[1,-1],[2,1],[5,2]])
m2 = np.array([[1,2,1,2],[4,1,-1,4]])

print("m1")
print(m1.shape)
print(m1)
print("m2")
print(m2.shape)
print(m2)
print("m1 x m2")
m3 = np.matmul(m1, m2)
print(m3.shape)
print(m3)

m1
(4, 2)
[[ 0  3]
 [ 1 -1]
 [ 2  1]
 [ 5  2]]
m2
(2, 4)
[[ 1  2  1  2]
 [ 4  1 -1  4]]
m1 x m2
(4, 4)
[[12  3 -3 12]
 [-3  1  2 -2]
 [ 6  5  1  8]
 [13 12  3 18]]


For 2.5 – 2.7, calculate by hand and then show how the results of using numpy.linalg.solve

# 2.5
Find the set $S$ of all solutions in $x$ of the following inhomogeneous linear systems $Ax = b$, where $A$ and $b$ are defined as follows:

## a.

$A=$
$
\begin{bmatrix}
1 & 1 & -1 & -1 \\
2 & 5 & -7 & -5 \\
2 & -1 & 1 & 3 \\
5 & 2 & -4 & 2
\end{bmatrix}
$
$,\:\: b=$
$
\begin{bmatrix}
1\\
-2\\
4\\
6
\end{bmatrix}
$

Using Gauss-Jordan to solve the equations.
</br></br>
Create the augmented matrix
</br></br>
$
\begin{bmatrix}
1 & 1 & -1 & -1 & 1\\
2 & 5 & -7 & -5 & -2\\
2 & -1 & 1 & 3 & 4\\
5 & 2 & -4 & 2 & 6
\end{bmatrix}
$
</br></br>
Multiply row one by 2
</br></br>
$
\begin{bmatrix}
2 & 2 & -2 & -2 & 2\\
2 & 5 & -7 & -5 & -2\\
2 & -1 & 1 & 3 & 4\\
5 & 2 & -4 & 2 & 6
\end{bmatrix}
$
</br></br>
Subtract row one from row two
</br></br>
$
\begin{bmatrix}
2 & 2 & -2 & -2 & 2\\
0 & 3 & -5 & -3 & -4\\
2 & -1 & 1 & 3 & 4\\
5 & 2 & -4 & 2 & 6
\end{bmatrix}
$
</br></br>
Subtract row one from row three
</br></br>
$
\begin{bmatrix}
2 & 2 & -2 & -2 & 2\\
0 & 3 & -5 & -3 & -4\\
0 & -3 & 3 & 5 & 2\\
5 & 2 & -4 & 2 & 6
\end{bmatrix}
$
</br></br>
Multiply row one by $\:\frac{1}{2}$
</br></br>
$
\begin{bmatrix}
1 & 1 & -1 & -1 & 1\\
0 & 3 & -5 & -3 & -4\\
0 & -3 & 3 & 5 & 2\\
5 & 2 & -4 & 2 & 6
\end{bmatrix}
$
</br></br>
Multiply row one by 5
</br></br>
$
\begin{bmatrix}
5 & 5 & -5 & -5 & 5\\
0 & 3 & -5 & -3 & -4\\
0 & -3 & 3 & 5 & 2\\
5 & 2 & -4 & 2 & 6
\end{bmatrix}
$
</br></br>
Subtract row one from row four
</br></br>
$
\begin{bmatrix}
5 & 5 & -5 & -5 & 5\\
0 & 3 & -5 & -3 & -4\\
0 & -3 & 3 & 5 & 2\\
0 & -3 & 1 & 7 & 1
\end{bmatrix}
$
</br></br>
Multiply row one by $\:\frac{1}{5}$
</br></br>
$
\begin{bmatrix}
1 & 1 & -1 & -1 & 1\\
0 & 3 & -5 & -3 & -4\\
0 & -3 & 3 & 5 & 2\\
0 & -3 & 1 & 7 & 1
\end{bmatrix}
$
</br></br>
Multiply row two by $\:\frac{1}{3}$
</br></br>
$
\begin{bmatrix}
1 & 1 & -1 & -1 & 1\\
0 & 1 & -\frac{5}{3} & -1 & -\frac{4}{3}\\
0 & -3 & 3 & 5 & 2\\
0 & -3 & 1 & 7 & 1
\end{bmatrix}
$
</br></br>
Subtract row two from row one
</br></br>
$
\begin{bmatrix}
1 & 0 & \frac{2}{3} & 0 & \frac{7}{3}\\
0 & 1 & -\frac{5}{3} & -1 & -\frac{4}{3}\\
0 & -3 & 3 & 5 & 2\\
0 & -3 & 1 & 7 & 1
\end{bmatrix}
$
</br></br>
Multiply row two by $\:-3$
</br></br>
$
\begin{bmatrix}
1 & 0 & \frac{2}{3} & 0 & \frac{7}{3}\\
0 & -3 & 5 & 3 & 4\\
0 & -3 & 3 & 5 & 2\\
0 & -3 & 1 & 7 & 1
\end{bmatrix}
$
</br></br>
Subtract row two from row three
</br></br>
$
\begin{bmatrix}
1 & 0 & \frac{2}{3} & 0 & \frac{7}{3}\\
0 & -3 & 5 & 3 & 4\\
0 & 0 & -2 & 2 & -2\\
0 & -3 & 1 & 7 & 1
\end{bmatrix}
$
</br></br>
Subtract row two from row four
</br></br>
$
\begin{bmatrix}
1 & 0 & \frac{2}{3} & 0 & \frac{7}{3}\\
0 & -3 & 5 & 3 & 4\\
0 & 0 & -2 & 2 & -2\\
0 & 0 & -4 & 4 & -3
\end{bmatrix}
$
</br></br>
Multiply row two by $\:-\frac{1}{3}$
</br></br>
$
\begin{bmatrix}
1 & 0 & \frac{2}{3} & 0 & \frac{7}{3}\\
0 & 1 & -\frac{5}{3} & -1 & -\frac{4}{3}\\
0 & 0 & -2 & 2 & -2\\
0 & 0 & -4 & 4 & -3
\end{bmatrix}
$
</br></br>
Multiply row three by $\:-\frac{1}{2}$
</br></br>
$
\begin{bmatrix}
1 & 0 & \frac{2}{3} & 0 & \frac{7}{3}\\
0 & 1 & -\frac{5}{3} & -1 & -\frac{4}{3}\\
0 & 0 & 1 & -1 & 1\\
0 & 0 & -4 & 4 & -3
\end{bmatrix}
$
</br></br>
Multiply row three by $\:\frac{2}{3}$
</br></br>
$
\begin{bmatrix}
1 & 0 & \frac{2}{3} & 0 & \frac{7}{3}\\
0 & 1 & -\frac{5}{3} & -1 & -\frac{4}{3}\\
0 & 0 & \frac{2}{3} & -\frac{2}{3} & \frac{2}{3}\\
0 & 0 & -4 & 4 & -3
\end{bmatrix}
$
</br></br>
Subtract row three from row one
</br></br>
$
\begin{bmatrix}
1 & 0 & 0 & \frac{2}{3} & \frac{5}{3}\\
0 & 1 & -\frac{5}{3} & -1 & -\frac{4}{3}\\
0 & 0 & \frac{2}{3} & -\frac{2}{3} & \frac{2}{3}\\
0 & 0 & -4 & 4 & -3
\end{bmatrix}
$
</br></br>
Multiply row three by $\:3$
</br></br>
$
\begin{bmatrix}
1 & 0 & 0 & \frac{2}{3} & \frac{5}{3}\\
0 & 1 & -\frac{5}{3} & -1 & -\frac{4}{3}\\
0 & 0 & 1 & -1 & 1\\
0 & 0 & -4 & 4 & -3
\end{bmatrix}
$
</br></br>
Multiply row three by  $\:-\frac{5}{3}$
</br></br>
$
\begin{bmatrix}
1 & 0 & 0 & \frac{2}{3} & \frac{5}{3}\\
0 & 1 & -\frac{5}{3} & -1 & -\frac{4}{3}\\
0 & 0 & -\frac{5}{3} & \frac{5}{3} & -\frac{5}{3}\\
0 & 0 & -4 & 4 & -3
\end{bmatrix}
$
</br></br>
Subtract row three from row two
</br></br>
$
\begin{bmatrix}
1 & 0 & 0 & \frac{2}{3} & \frac{5}{3}\\
0 & 1 & 0 & -\frac{8}{3} & \frac{1}{3}\\
0 & 0 & -\frac{5}{3} & \frac{5}{3} & -\frac{5}{3}\\
0 & 0 & -4 & 4 & -3
\end{bmatrix}
$
</br></br>
Multiply row three by $\:-\frac{3}{5}$
</br></br>
$
\begin{bmatrix}
1 & 0 & 0 & \frac{2}{3} & \frac{5}{3}\\
0 & 1 & 0 & -\frac{8}{3} & \frac{1}{3}\\
0 & 0 & 1 & -1 & 1\\
0 & 0 & -4 & 4 & -3
\end{bmatrix}
$
</br></br>
Multiply row three by $\:-4$
</br></br>
$
\begin{bmatrix}
1 & 0 & 0 & \frac{2}{3} & \frac{5}{3}\\
0 & 1 & 0 & -\frac{8}{3} & \frac{1}{3}\\
0 & 0 & -4 & 4 & -4\\
0 & 0 & -4 & 4 & -3
\end{bmatrix}
$
</br></br>
Subtract row three from row four
</br></br>
$
\begin{bmatrix}
1 & 0 & 0 & \frac{2}{3} & \frac{5}{3}\\
0 & 1 & 0 & -\frac{8}{3} & \frac{1}{3}\\
0 & 0 & -4 & 4 & -4\\
0 & 0 & 0 & 0 & 1
\end{bmatrix}
$
</br></br>
We now have all zeros in row four so the system is inconsistent. There is no solution.

In [7]:
A = np.array([[1,1,-1,-1],[2,5,-7,-5],[2,-1,1,3],[5,2,-4,2]])
b = np.array([[1],[-2],[4],[6]])

print("A")
print(A.shape)
print(A)
print("b")
print(b.shape)
print(b)
print("Solving Ax = b")
sol = np.linalg.solve(A,b)
print(sol.shape)
print(sol)

A
(4, 4)
[[ 1  1 -1 -1]
 [ 2  5 -7 -5]
 [ 2 -1  1  3]
 [ 5  2 -4  2]]
b
(4, 1)
[[ 1]
 [-2]
 [ 4]
 [ 6]]
Solving Ax = b
(4, 1)
[[ 7.50599938e+14]
 [-3.00239975e+15]
 [-1.12589991e+15]
 [-1.12589991e+15]]


Why could Numpy find a solution? It must be using a numerical method that will approximate a solution. The numbers are very small.

## b.

$A=$
$
\begin{bmatrix}
1 & -1 & 0 & 0 & 1 \\
1 & 1 & 0 & -3 & 0 \\
2 & -1 & 0 & 1 & -1 \\
-1 & 2 & 0 & -2 & -1
\end{bmatrix}
$
$,\:\: b=$
$
\begin{bmatrix}
3\\
6\\
5\\
-1
\end{bmatrix}
$

Using Gauss-Jordan to solve the equations.
</br></br>
Create the augmented matrix
</br></br>
$\begin{bmatrix}
 1 & -1 & 0 &  0 &  1 &  3\\
 1 &  1 & 0 & -3 &  0 &  6\\
 2 & -1 & 0 &  1 & -1 &  5\\
-1 &  2 & 0 & -2 & -1 & -1
\end{bmatrix}$
</br></br>Subtract row one from row two</br></br>
$\begin{bmatrix}
 1 & -1 & 0 &  0 &  1 &  3\\
 0 &  2 & 0 & -3 & -1 &  3\\
 2 & -1 & 0 &  1 & -1 &  5\\
-1 &  2 & 0 & -2 & -1 & -1
\end{bmatrix}$
</br></br>Subtract $2$ times the first row from the third row</br></br>
$\begin{bmatrix}
 1 & -1 & 0 &  0 &  1 &  3\\
 0 &  2 & 0 & -3 & -1 &  3\\
 0 &  1 & 0 &  1 & -3 & -1\\
-1 &  2 & 0 & -2 & -1 & -1
\end{bmatrix}$
</br></br>Subtract $-1$ times the first row from the fourth row</br></br>
$\begin{bmatrix}
 1 & -1 & 0 &  0 &  1 &  3\\
 0 &  2 & 0 & -3 & -1 &  3\\
 0 &  1 & 0 &  1 & -3 & -1\\
 0 &  1 & 0 & -2 &  0 &  2
\end{bmatrix}$
</br></br>Swap the second and third rows</br></br>
$\begin{bmatrix}
 1 & -1 & 0 &  0 &  1 &  3\\
 0 &  1 & 0 &  1 & -3 & -1\\
 0 &  2 & 0 & -3 & -1 &  3\\
 0 &  1 & 0 & -2 &  0 &  2
\end{bmatrix}$
</br></br>Subtract $-1$ times the second row from the first row</br></br>
$\begin{bmatrix}
 1 &  0 & 0 &  1 & -2 &  2\\
 0 &  1 & 0 &  1 & -3 & -1\\
 0 &  2 & 0 & -3 & -1 &  3\\
 0 &  1 & 0 & -2 &  0 &  2
\end{bmatrix}$
</br></br>Subtract $2$ times the second row from the third row</br></br>
$\begin{bmatrix}
 1 &  0 & 0 &  1 & -2 &  2\\
 0 &  1 & 0 &  1 & -3 & -1\\
 0 &  0 & 0 & -5 &  5 &  5\\
 0 &  1 & 0 & -2 &  0 &  2
\end{bmatrix}$
</br></br>Subtract the second row from the fourth row</br></br>
$\begin{bmatrix}
 1 &  0 & 0 &  1 & -2 &  2\\
 0 &  1 & 0 &  1 & -3 & -1\\
 0 &  0 & 0 & -5 &  5 &  5\\
 0 &  0 & 0 & -3 &  3 &  3
\end{bmatrix}$
</br></br>Divide the third row by $-5$</br></br>
$\begin{bmatrix}
 1 &  0 & 0 &  1 & -2 &  2\\
 0 &  1 & 0 &  1 & -3 & -1\\
 0 &  0 & 0 &  1 & -1 & -1\\
 0 &  0 & 0 & -3 &  3 &  3
\end{bmatrix}$
</br></br>Subtract the third row from the first row</br></br>
$\begin{bmatrix}
 1 &  0 & 0 &  0 & -1 &  3\\
 0 &  1 & 0 &  1 & -3 & -1\\
 0 &  0 & 0 &  1 & -1 & -1\\
 0 &  0 & 0 & -3 &  3 &  3
\end{bmatrix}$
</br></br>Subtract the third row from the second row</br></br>
$\begin{bmatrix}
 1 &  0 & 0 &  0 & -1 &  3\\
 0 &  1 & 0 &  0 & -2 &  0\\
 0 &  0 & 0 &  1 & -1 & -1\\
 0 &  0 & 0 & -3 &  3 &  3
\end{bmatrix}$
</br></br>Subtract $-3$ times the third row from the fourth row</br></br>
$\begin{bmatrix}
 1 &  0 & 0 &  0 & -1 &  3\\
 0 &  1 & 0 &  0 & -2 &  0\\
 0 &  0 & 0 &  1 & -1 & -1\\
 0 &  0 & 0 &  0 &  0 &  0
\end{bmatrix}$
</br></br>Reading from the matrix, the solution set is:</br></br>
$x_1 = 3+x_5$
</br>
$x_2 = 2x_5$
</br>
$x_3 \:\text{is free}$
</br>
$x_4 = -1+x_5$
</br>
$x_5 \:\text{is free}$

In [8]:
A = np.array([[1,-1,0,0,1],[1,1,0,-3,0],[2,-1,0,1,-1],[-1,2,0,-2,-1]])
b = np.array([[3],[6],[5],[-1]])

print("A")
print(A.shape)
print(A)
print("b")
print(b.shape)
print(b)
print("Solving Ax = b")
sol = np.linalg.solve(A,b)
print(sol.shape)
print(sol)

A
(4, 5)
[[ 1 -1  0  0  1]
 [ 1  1  0 -3  0]
 [ 2 -1  0  1 -1]
 [-1  2  0 -2 -1]]
b
(4, 1)
[[ 3]
 [ 6]
 [ 5]
 [-1]]
Solving Ax = b


LinAlgError: Last 2 dimensions of the array must be square

I suppose that Numpy is using a numerical method that can only solve for one value and therefore does not work in this case.

# 2.6

Use Gaussian Elimination, find all solutions of the inhomogeneous equation system $Ax=b$ with

$A=$
$
\begin{bmatrix}
0 & 1 & 0 & 0 & 1 & 0\\
0 & 0 & 0 & 1 & 1 & 0\\
0 & 1 & 0 & 0 & 0 & 1
\end{bmatrix}
$
$,\:\: b=$
$
\begin{bmatrix}
2\\
-1\\
1
\end{bmatrix}
$

Using Gauss-Jordan to solve the equations.
</br></br>
Create the augmented matrix
</br></br>
$\begin{bmatrix}
 0 &  1 & 0 &  0 &  1 &  0 &  2\\
 0 &  0 & 0 &  1 &  1 &  0 & -1\\
 0 &  1 & 0 &  0 &  0 &  1 &  1
\end{bmatrix}$
</br></br>Subtract row one from row three</br></br>
$\begin{bmatrix}
 0 &  1 & 0 &  0 &  1 &  0 &  2\\
 0 &  0 & 0 &  1 &  1 &  0 & -1\\
 0 &  0 & 0 &  0 & -1 &  1 & -1
\end{bmatrix}$
</br></br>Multiply row three by $-1$</br></br>
$\begin{bmatrix}
 0 &  1 & 0 &  0 &  1 &  0 &  2\\
 0 &  0 & 0 &  1 &  1 &  0 & -1\\
 0 &  0 & 0 &  0 &  1 & -1 &  1
\end{bmatrix}$
</br></br>Subtract row three from row one</br></br>
$\begin{bmatrix}
 0 &  1 & 0 &  0 &  0 &  1 &  1\\
 0 &  0 & 0 &  1 &  1 &  0 & -1\\
 0 &  0 & 0 &  0 &  1 & -1 &  1
\end{bmatrix}$
</br></br>Subtract row three from row two</br></br>
$\begin{bmatrix}
 0 &  1 & 0 &  0 &  0 &  1 &  1\\
 0 &  0 & 0 &  1 &  0 &  1 & -2\\
 0 &  0 & 0 &  0 &  1 & -1 &  1
\end{bmatrix}$
</br></br>Reading from the matrix, the solution set is:</br></br>
$x_1 \:\text{is free}$
</br>
$x_2 = 1-x_6$
</br>
$x_3 \:\text{is free}$
</br>
$x_4 = -2-x_6$
</br>
$x_5 = 1+x_6$
</br>
$x_6 \:\text{is free}$

In [9]:
A = np.array([[0,1,0,0,1,0],[0,0,0,1,1,0],[0,1,0,0,0,1]])
b = np.array([[2],[-1],[1]])

print("A")
print(A.shape)
print(A)
print("b")
print(b.shape)
print(b)
print("Solving Ax = b")
sol = np.linalg.solve(A,b)
print(sol.shape)
print(sol)

A
(3, 6)
[[0 1 0 0 1 0]
 [0 0 0 1 1 0]
 [0 1 0 0 0 1]]
b
(3, 1)
[[ 2]
 [-1]
 [ 1]]
Solving Ax = b


LinAlgError: Last 2 dimensions of the array must be square

# 2.7

Find all solutions in $x = $ $\begin{bmatrix}x_1\\ x_2\\ x_3 \end{bmatrix} \in \mathbb{R}^3$  of the equation system $Ax = 12x$, where

$A=$
$
\begin{bmatrix}
6 & 4 & 3\\
6 & 0 & 9\\
0 & 8 & 0
\end{bmatrix}
$
</br></br>
and $ \displaystyle\sum_{i=1}^{3} x_i=1 $

Given
</br></br>
$A=$
$
\begin{bmatrix}
6 & 4 & 3\\
6 & 0 & 9\\
0 & 8 & 0
\end{bmatrix}
$
</br></br>
$x = $
$\begin{bmatrix}x_1\\ x_2\\ x_3 \end{bmatrix}$
</br></br>
$Ax = 12x$
</br></br>
and $\displaystyle\sum_{i=1}^{3} x_i=1 $
</br></br>
then
</br></br>
$\frac{1}{12}Ax = x$
</br></br>
$\frac{1}{12}
\begin{bmatrix}
6 & 4 & 3\\
6 & 0 & 9\\
0 & 8 & 0
\end{bmatrix}\begin{bmatrix}x_1\\ x_2\\ x_3 \end{bmatrix} = \begin{bmatrix}x_1\\ x_2\\ x_3 \end{bmatrix}$
</br></br>
$\begin{bmatrix}
\frac{1}{2} & \frac{1}{3} & \frac{1}{4}\\
\frac{1}{2} & 0 & \frac{3}{4}\\
0 & \frac{2}{3} & 0
\end{bmatrix}\begin{bmatrix}x_1\\ x_2\\ x_3 \end{bmatrix} = \begin{bmatrix}x_1\\ x_2\\ x_3 \end{bmatrix}$
</br></br>
From row three of the matrix
</br></br>
$\frac{2}{3}x_2$ = $x_3$
</br></br>
From row one of the matrix
</br></br>
$\frac{1}{2}x_1 + \frac{1}{3}x_2 + \frac{1}{4}x_3 = x_1$
</br></br>
Substituting for $x_3$
</br></br>
$\frac{1}{2}x_1 + \frac{1}{3}x_2 + \frac{1}{4}\frac{2}{3}x_2 = x_1$
</br></br>
Calculating
</br></br>
$\frac{1}{2}x_1 + \frac{1}{3}x_2 + \frac{2}{12}x_2 = x_1$
</br></br>
Multiply term two by $\frac{4}{4}$
</br></br>
$\frac{1}{2}x_1 + \frac{4}{12}x_2 + \frac{2}{12}x_2 = x_1$
</br></br>
Combine terms two and three
</br></br>
$\frac{1}{2}x_1 + \frac{6}{12}x_2 = x_1$
</br></br>
Simplify
</br></br>
$\frac{1}{2}x_1 + \frac{1}{2}x_2 = x_1$
</br></br>
Combine terms one and two
</br></br>
$\frac{1}{2}x_2 = \frac{1}{2}x_1$
</br></br>
Multiply both sides by  2
</br></br>
$x_2 = x_1$
</br></br>
We now have $x_1 = x_2$ and $x_3 = \frac{2}{3}x_2$. We also know that $\displaystyle\sum_{i=1}^{3} x_i=1 $.
</br></br>
Expand the sum
</br></br>
$x_1 + x_2 + x_3 = 1$
</br></br>
Substitute for $x_1$ and $x_3$
</br></br>
$x_2 + x_2 + \frac{2}{3}x_2 = 1$
</br></br>
Combine terms one, two, and three
</br></br>
$2\frac{2}{3}x_2 = 1$
</br></br>
Express as a fraction
</br></br>
$\frac{8}{3}x_2 = 1$
</br></br>
Solve for $x_2$
</br></br>
$x_2 = \frac{3}{8}$
</br></br>
Therefore
</br></br>
$x_1 = \frac{3}{8}$
</br></br>
$x_2 = \frac{3}{8}$
</br></br>
$x_3 = \frac{2}{8}$

This seems to be the only solution. I cannot think of a way to present this to Numpy to calculate.

For 2.8, calculate by hand and then show the results of using numpy.linalg.inv

# 2.8

Determine the inverses of the following matrices if possible:

## a.
$A=$
$
\begin{bmatrix}
2 & 3 & 4\\
3 & 4 & 5\\
4 & 5 & 6
\end{bmatrix}
$

Attempting to invert the matrix by augmenting it with an Identity Matrix and then performing gaussian elimination.
</br></br>
Append the identity matrix
</br></br>
$
\begin{bmatrix}
2 & 3 & 4 & 1 & 0 & 0\\
3 & 4 & 5 & 0 & 1 & 0\\
4 & 5 & 6 & 0 & 0 & 1
\end{bmatrix}
$
</br></br>
Swap rows 2 and 3
</br></br>
$
\begin{bmatrix}
2 & 3 & 4 & 1 & 0 & 0\\
4 & 5 & 6 & 0 & 0 & 1\\
3 & 4 & 5 & 0 & 1 & 0
\end{bmatrix}
$
</br></br>
Subtract 2 * Row 1 from Row 2
</br></br>
$
\begin{bmatrix}
2 & 3 & 4 & 1 & 0 & 0\\
0 & -1 & -2 & -2 & 0 & 1\\
3 & 4 & 5 & 0 & 1 & 0
\end{bmatrix}
$
</br></br>
Multiply row 3 by 2
</br></br>
$
\begin{bmatrix}
2 & 3 & 4 & 1 & 0 & 0\\
0 & -1 & -2 & -2 & 0 & 1\\
6 & 8 & 10 & 0 & 2 & 0
\end{bmatrix}
$
</br></br>
Subtract 3 times row 1 from row 3
</br></br>
$
\begin{bmatrix}
2 & 3 & 4 & 1 & 0 & 0\\
0 & -1 & -2 & -2 & 0 & 1\\
0 & -1 & -2 & -3 & 2 & 0
\end{bmatrix}
$
</br></br>
Subtract row 2 from row 3
</br></br>
$
\begin{bmatrix}
2 & 3 & 4 & 1 & 0 & 0\\
0 & -1 & -2 & -2 & 0 & 1\\
0 & 0 & 0 & -1 & 2 & -1
\end{bmatrix}
$
</br></br>
The inverse is
</br></br>
There is no inverse since the two rows had the same values, the matrix is singular.

In [10]:
# In code
A = np.array([[2,3,4],[3,4,5],[4,5,6]])

print("A")
print(A.shape)
print(A)
print("Calculating the inverse of A")
sol = np.linalg.inv(A)
print(sol.shape)
print(sol)

A
(3, 3)
[[2 3 4]
 [3 4 5]
 [4 5 6]]
Calculating the inverse of A


LinAlgError: Singular matrix

My results and Numpy agree that this matrix has no inverse.

## b.
$A=$
$
\begin{bmatrix}
1 & 0 & 1 & 0\\
0 & 1 & 1 & 0\\
1 & 1 & 0 & 1\\
1 & 1 & 1 & 0
\end{bmatrix}
$

Attempting to invert the matrix by augmenting it with an Identity Matrix and then performing gaussian elimination.
</br></br>
Append the identity matrix
</br></br>
$
\begin{bmatrix}
1 & 0 & 1 & 0 & 1 & 0 & 0 & 0\\
0 & 1 & 1 & 0 & 0 & 1 & 0 & 0\\
1 & 1 & 0 & 1 & 0 & 0 & 1 & 0\\
1 & 1 & 1 & 0 & 0 & 0 & 0 & 1
\end{bmatrix}
$
</br></br>
Subtract row 1 from row 3
</br></br>
$
\begin{bmatrix}
1 & 0 & 1 & 0 & 1 & 0 & 0 & 0\\
0 & 1 & 1 & 0 & 0 & 1 & 0 & 0\\
0 & 1 & -1 & 1 & -1 & 0 & 1 & 0\\
1 & 1 & 1 & 0 & 0 & 0 & 0 & 1
\end{bmatrix}
$
</br></br>
Subtract row 1 from row 4
</br></br>
$
\begin{bmatrix}
1 & 0 & 1 & 0 & 1 & 0 & 0 & 0\\
0 & 1 & 1 & 0 & 0 & 1 & 0 & 0\\
0 & 1 & -1 & 1 & -1 & 0 & 1 & 0\\
0 & 1 & 0 & 0 & -1 & 0 & 0 & 1
\end{bmatrix}
$
</br></br>
Subtract row 2 from row 3
</br></br>
$
\begin{bmatrix}
1 & 0 & 1 & 0 & 1 & 0 & 0 & 0\\
0 & 1 & 1 & 0 & 0 & 1 & 0 & 0\\
0 & 0 & -2 & 1 & -1 & -1 & 1 & 0\\
0 & 1 & 0 & 0 & -1 & 0 & 0 & 1
\end{bmatrix}
$
</br></br>
Subtract row 2 from row 4
</br></br>
$
\begin{bmatrix}
1 & 0 & 1 & 0 & 1 & 0 & 0 & 0\\
0 & 1 & 1 & 0 & 0 & 1 & 0 & 0\\
0 & 0 & -2 & 1 & -1 & -1 & 1 & 0\\
0 & 0 & -1 & 0 & -1 & -1 & 0 & 1
\end{bmatrix}
$
</br></br>
Swap rows 3 and 4
</br></br>
$
\begin{bmatrix}
1 & 0 & 1 & 0 & 1 & 0 & 0 & 0\\
0 & 1 & 1 & 0 & 0 & 1 & 0 & 0\\
0 & 0 & -1 & 0 & -1 & -1 & 0 & 1\\
0 & 0 & -2 & 1 & -1 & -1 & 1 & 0
\end{bmatrix}
$
</br></br>
Multiply row 3 by -1
</br></br>
$
\begin{bmatrix}
1 & 0 & 1 & 0 & 1 & 0 & 0 & 0\\
0 & 1 & 1 & 0 & 0 & 1 & 0 & 0\\
0 & 0 & 1 & 0 & 1 & 1 & 0 & -1\\
0 & 0 & -2 & 1 & -1 & -1 & 1 & 0
\end{bmatrix}
$
</br></br>
Subtract row 3 from row 1
</br></br>
$
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 & -1 & 0 & 1\\
0 & 1 & 1 & 0 & 0 & 1 & 0 & 0\\
0 & 0 & 1 & 0 & 1 & 1 & 0 & -1\\
0 & 0 & -2 & 1 & -1 & -1 & 1 & 0
\end{bmatrix}
$
</br></br>
Subtract row 3 from row 2
</br></br>
$
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 & -1 & 0 & 1\\
0 & 1 & 0 & 0 & -1 & 0 & 0 & 1\\
0 & 0 & 1 & 0 & 1 & 1 & 0 & -1\\
0 & 0 & -2 & 1 & -1 & -1 & 1 & 0
\end{bmatrix}
$
</br></br>
Multiply row 3 by $-2$
</br></br>
$
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 & -1 & 0 & 1\\
0 & 1 & 0 & 0 & -1 & 0 & 0 & 1\\
0 & 0 & -2 & 0 & -2 & -2 & 0 & 2\\
0 & 0 & -2 & 1 & -1 & -1 & 1 & 0
\end{bmatrix}
$
</br></br>
Subtract row 3 from row 4
</br></br>
$
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 & -1 & 0 & 1\\
0 & 1 & 0 & 0 & -1 & 0 & 0 & 1\\
0 & 0 & -2 & 0 & -2 & -2 & 0 & 2\\
0 & 0 & 0 & 1 & 1 & 1 & 1 & -2
\end{bmatrix}
$
</br></br>
Multiply row 3 by $\:-\frac{1}{2}$
</br></br>
$
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 & -1 & 0 & 1\\
0 & 1 & 0 & 0 & -1 & 0 & 0 & 1\\
0 & 0 & 1 & 0 & 1 & 1 & 0 & -1\\
0 & 0 & 0 & 1 & 1 & 1 & 1 & -2
\end{bmatrix}
$
</br></br>
The inverse is
</br></br>
$
\begin{bmatrix}
0 & -1 & 0 & 1\\
-1 & 0 & 0 & 1\\
1 & 1 & 0 & -1\\
1 & 1 & 1 & -2
\end{bmatrix}
$

In [11]:
# In code
A = np.array([[1,0,1,0],[0,1,1,0],[1,1,0,1],[1,1,1,0]])

print("A")
print(A.shape)
print(A)
print("Calculating the inverse of A")
sol = np.linalg.inv(A)
print(sol.shape)
print(sol)

A
(4, 4)
[[1 0 1 0]
 [0 1 1 0]
 [1 1 0 1]
 [1 1 1 0]]
Calculating the inverse of A
(4, 4)
[[ 0. -1.  0.  1.]
 [-1.  0.  0.  1.]
 [ 1.  1. -0. -1.]
 [ 1.  1.  1. -2.]]


The results match. It actually took me multiple tries to get to a solution.